# Libraries

In [40]:
from pathlib import Path
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
import shutil
import pandas as pd

In [41]:
notebooks_path = Path.cwd()
repo_path = notebooks_path.parent
print(f'current directory is: {notebooks_path}')

import utils_ric as utils
from info import patient

current directory is: /home/ricardino/Documents/MAIA/tercer_semestre/MIRA/final_project/MIRA_FINAL_PROJECT/notebooks


# Functions

In [46]:
def image_registration(fixed_image, moving_image, param_file=None):
    """Give two images and they will be registered. The outputs are the moving image registered the transformation map.

    Args:
        fixed_image (sitk image): fixed (template) image
        moving_image (sitk image): moving image (image that will be transformed)

    Returns:
        sitk image, transformix map: transformed image and the transformation map
    """
    #Start registration settings
    elastixImageFilter = sitk.ElastixImageFilter() #Image filter object
    #Set parameter file
    if param_file is not None: #if the user gives a parameter file
        elastixImageFilter.SetParameterMap(param_file)
    #Set images
    elastixImageFilter.SetFixedImage(fixed_image)
    elastixImageFilter.SetMovingImage(moving_image)

    #Run registration
    elastixImageFilter.Execute()

    #Get result image
    resultImage = elastixImageFilter.GetResultImage()

    #Transformation map
    transformParameterMap = elastixImageFilter.GetTransformParameterMap()
    
    return resultImage, transformParameterMap

In [42]:
def register_points(pat, transformParameterMap, points_path):
    """Register points using the transformation map
        IMPORTANT: the points must be in the same space as the moving image
            - This happens because the transformation maps the moving image to the fixed image
            - This is contrary to what usually happens in image registration
            - See SimpleElastix documentation for more information

    Args:
        transformParameterMap (transformix map): transformation map
        points_path (str): path to pts file (txt file with points)
        moving_image (sitk image): moving image (reference image)
        pat (obj): patient object

    Returns:
        Nothing: the points are saved in a txt file
    """
    #Transformix filter object
    transformixImageFilter = sitk.TransformixImageFilter()
    #Set transformation map
    transformixImageFilter.SetTransformParameterMap(transformParameterMap)
    #Set points
    transformixImageFilter.SetFixedPointSetFileName(str(repo_path / points_path))
    #Set moving image (needed to get spacial information)
    transformixImageFilter.SetMovingImage(pat.im_sitk('e')) 
    #Run transformation
    transformixImageFilter.Execute()
    
    #change name and relocate outputpoints.txt file
    old_name = notebooks_path / 'outputpoints.txt'
    new_name = repo_path / 'data/transformed_keypoints' / f'outputpoints_pat{pat.pat_num}_trans-default.txt'
    shutil.move(old_name, new_name)

In [43]:
def read_outputpoints(points_path):
    """Read the outputpoints.txt file and return the coordinates of the points
    
        args:
            points_path (str): path to the outputpoints.txt file
        returns:
            numpy array: array with the coordinates of the points
    """ 
    df = pd.read_csv(points_path, sep="\t", header=None)
    #get column 5 (where the output points are)
    outpoints = df.iloc[:, 5]
    #remove text to keep just x y z coordinates
    outpoints = outpoints.str.replace('; OutputPoint = \[ ', '', regex=True)
    outpoints = outpoints.str.replace(' \]', '', regex=True)
    #transform to numpy the series with the coordinates separated by \tab
    outpoints = outpoints.str.split(' ', expand=True).to_numpy(dtype=float)
    return abs(outpoints) #absolute value because the z coordinates are negative (we inverted the axis for visualization)

# MAIN

## Parameter files

The parameter files are the main component of the registration process. They contain all the information needed to perform the registration. The parameter files are written in a txt file according to the Elastix documentation.

In [57]:
def save_paramfile(elastixImageFilter, name):
    
    param_vector = elastixImageFilter.GetParameterMap()
    for i in range(len(param_vector)):
        list_param = param_vector[i].items()
        path_file = repo_path / 'data/parameterfiles' / f'{name}_{i}.txt'
        with open(path_file, 'w') as f:
            for line in list_param:
                f.write(f"{line}\n")

In [125]:
elastixImageFilter = sitk.ElastixImageFilter() #Image filter object
param_vector = elastixImageFilter.GetParameterMap()
param_list = list(param_vector[0].items())
df = pd.DataFrame(param_list)


dtype('O')

In [45]:
# #Image filter object
# elastixImageFilter = sitk.ElastixImageFilter()
# #Save dafualt parameter map
# save_paramfile(elastixImageFilter, name='default')

"('AutomaticParameterEstimation', ('true',))"

## Image registration

Now we can register the images. We just have to give the fixed image, moving image and the parameter file.

In [47]:
#Define patient
pat = patient(num=1)
#Get param map file
param_file =sitk.ReadParameterFile(str(repo_path / 'data/parameterfiles' / 'best_bspline_par.txt'))
_, transformParameterMap = image_registration(fixed_image=pat.im_sitk('i'), moving_image = pat.im_sitk('e'), param_file=param_file)

Installing all components.
InstallingComponents was successful.

ELASTIX version: 5.000
Command line options from ElastixBase:
-fMask    unspecified, so no fixed mask used
-mMask    unspecified, so no moving mask used
-out      ./
-threads  unspecified, so all available threads are used
Command line options from TransformBase:
-t0       unspecified, so no initial transform used
  The default value "3" is used instead.
  The default value "false" is used instead.

Reading images...
Reading images took 0 ms.

Initialization of all components (before registration) took: 0 ms.
Preparation of the image pyramids took: 6058 ms.

Resolution: 0
  The default value "true" is used instead.
  The default value "true" is used instead.
Setting the fixed masks took: 0 ms.
Setting the moving masks took: 0 ms.
  The default value "1" is used instead.
  The default value "32" is used instead.
  The default value "32" is used instead.
  The default value "32" is used instead.
  The default value "0.01" i

In [48]:
#transform the points. They will be saved in a txt file
register_points(pat, transformParameterMap, points_path = pat.points_path(type='i', format='pts'))

ELASTIX version: 5.000
Command line options from ElastixBase:
-out      ./
-threads  unspecified, so all available threads are used
-def      /home/ricardino/Documents/MAIA/tercer_semestre/MIRA/final_project/MIRA_FINAL_PROJECT/data/keypoints/copd1_300_iBH_xyz_r1.pts
-jac      unspecified, so no det(dT/dx) computed
-jacmat   unspecified, so no dT/dx computed

Reading input image ...
  Reading input image took 0.000001 s
Calling all ReadFromFile()'s ...
  The default value "false" is used instead.
  Calling all ReadFromFile()'s took 0.057348 s
Transforming points ...
  The transform is evaluated on some points, specified in the input point file.
  Reading input point file: /home/ricardino/Documents/MAIA/tercer_semestre/MIRA/final_project/MIRA_FINAL_PROJECT/data/keypoints/copd1_300_iBH_xyz_r1.pts
  Input points are specified as image indices.
  Number of specified input points: 300
  The input points are transformed.
  The transformed points are saved in: ./outputpoints.txt
  Transforming

In [56]:
#read txt file with transformed points
points_path = repo_path / f'data/transformed_keypoints/outputpoints_pat{pat.pat_num}_trans-default.txt'
outpoints = read_outputpoints(points_path)
exhale_points = pat.get_landmark('e')
tre = utils.calculate_tre(outpoints, exhale_points)
#mean and std of tre
print(f'mean tre: {np.mean(tre)}')
print(f'std tre: {np.std(tre)}')

mean tre: 5.496071278195915
std tre: 5.067591762839297


,patient,mean_tre,std_tre
0,1,5.496071,5.067592
